<a href="https://colab.research.google.com/github/Vektor79/Zadania_samodzielne/blob/main/bakery_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
from google.colab import drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Sprzedaż w piekarni The Bread Basket



W tym zeszycie zajmiemy się wizualizacją danych dotyczących sprzedaży w latach 2016-2017 w piekarni *The Bread Basket* w Edynburgu.

Dane pochodzą ze zbioru udostępnionego [na stronie kaggle](https://www.kaggle.com/akashdeepkuila/bakery) w ramach licencji *CC0*.

Zapoznajmy się z materiałem dostępnym w notebooku i na podstawie instrukcji przygotujmy dane, które będą potrzebne podczas zajęć.

**Uwaga.** Wracając do dokumentu, pamiętaj o ponownym wywołaniu komórek z kodem.

### Zawartość zbioru danych

W zaimportowanym niżej pliku **bakery_sales.csv** znajduje się 20 507 pozycji przypisanych do 9 684 transakcji klientów z informacjami o:


*   **TransactionNo** - numerze transakcji
*   **Items** - zakupionym produkcie
*   **DateTime** - czasie transakcji
*   **Daypart** - porze dnia (rano, po południu, wieczorem, w nocy)
*   **DayType** - typ dnia (dzień roboczy vs weekend).

Potrzebne nam dane zostały udostępnione wraz z dokumentem - poniższy kod importuje potrzebne pliki.

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Dane importujemy do ramki danych pandas, dzięki czemu będziemy mogli sprawnie pracować nad ich wizualizacją.

In [6]:
bakery_data = pd.read_csv('/content/drive/My Drive/Vis/01_Bakery_Data/bakery_sales.csv')
bakery_data

,TransactionNo,Items,DateTime,Daypart,DayType
0,1,Bread,30/10/2016 9:58,Morning,Weekend
1,2,Scandinavian,30/10/2016 10:05,Morning,Weekend
2,2,Scandinavian,30/10/2016 10:05,Morning,Weekend
3,3,Hot chocolate,30/10/2016 10:07,Morning,Weekend
4,3,Jam,30/10/2016 10:07,Morning,Weekend
...,...,...,...,...,...
20502,9682,Coffee,04/09/2017 14:32,Afternoon,Weekend
20503,9682,Tea,04/09/2017 14:32,Afternoon,Weekend
20504,9683,Coffee,04/09/2017 14:57,Afternoon,Weekend
20505,9683,Pastry,04/09/2017 14:57,Afternoon,Weekend


## Przygotowanie zeszytu

Na początku chcemy upewnić się, że dane zostały poprawnie rozpoznane i dokonać potrzebnych konwersji.

Na podstawie poglądu danych powyżej spodziewamy się, że pierwsza kolumna zawiera informację w postaci kolejnych liczb całkowitych, w kolumnie drugiej znajdziemy nazwy różnych sprzedawanych produktów, kolumna trzecia powinna zawierać dane rozpoznane jako data z czasem, zaś ostatnie dwie kolumny powinny zawierać informacje w postaci tekstowej.

### Sprawdzenie typów danych

Poniżej potrzebujemy wykonać dołączone instrukcje, aby otrzymać ramki danych wykorzystywane na zajęciach.

Na początek sprawdźmy jak zostały rozpoznane dane podczas importu.

In [7]:
bakery_data.dtypes

,0
TransactionNo,int64
Items,object
DateTime,object
Daypart,object
DayType,object


Upewnijmy się, czy w danych znajdują się rekordy, dla których brakuje informacji w którejkolwiek z kolumn.

In [8]:
"complete records: " + str(len(bakery_data.dropna(how="any"))) + "; total records: " + str(len(bakery_data))

'complete records: 20507; total records: 20507'

Przyjrzymy się jeszcze temu, jakie dane faktycznie ukrywają się pod typem **object** w przypadku każdej z kolumn.

In [9]:
for column in bakery_data.columns:
  check_types = bakery_data[column].apply(lambda x: type(x))
  print(check_types.value_counts())

TransactionNo
<class 'int'>    20507
Name: count, dtype: int64
Items
<class 'str'>    20507
Name: count, dtype: int64
DateTime
<class 'str'>    20507
Name: count, dtype: int64
Daypart
<class 'str'>    20507
Name: count, dtype: int64
DayType
<class 'str'>    20507
Name: count, dtype: int64


#### Konwersja daty

W przypadku czasu transakcji domyślnie jest ona identyfikowana jako *string*.

Zamieńmy dane w kolumnie **DateTime** na  *timestamp*.

In [10]:
bakery_data["DateTime"] = pd.to_datetime(bakery_data["DateTime"])

<ipython-input-10-f9936c4c7655>:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  bakery_data["DateTime"] = pd.to_datetime(bakery_data["DateTime"])


Dodajmy nową kolumnę z datą transakcji o nazwie **Date** i sprawdźmy poprawność konwersji.

In [11]:
bakery_data["Date"] = bakery_data["DateTime"].dt.date

In [12]:
bakery_data["Date"].value_counts()

,count
Date,
2017-04-02,292
2016-05-11,275
2017-04-03,257
2017-03-25,246
2017-01-28,237
...,...
2017-09-04,69
2017-01-16,66
2016-08-12,66


Ponieważ nie będziemy wykorzystywać informacji o czasie, pozostawmy w **bakery_data** jedynie kolumnę z datą.

In [13]:
bakery_data = bakery_data[["TransactionNo", "Items", "Date", "Daypart", "DayType"]]
bakery_data

,TransactionNo,Items,Date,Daypart,DayType
0,1,Bread,2016-10-30,Morning,Weekend
1,2,Scandinavian,2016-10-30,Morning,Weekend
2,2,Scandinavian,2016-10-30,Morning,Weekend
3,3,Hot chocolate,2016-10-30,Morning,Weekend
4,3,Jam,2016-10-30,Morning,Weekend
...,...,...,...,...,...
20502,9682,Coffee,2017-09-04,Afternoon,Weekend
20503,9682,Tea,2017-09-04,Afternoon,Weekend
20504,9683,Coffee,2017-09-04,Afternoon,Weekend
20505,9683,Pastry,2017-09-04,Afternoon,Weekend


#### Przypisanie kategorii bazując na liczbie sprzedanych produktów

Przyjrzymy się bliżej zawartości kolumny **Items**.

In [14]:
bakery_data["Items"].value_counts()

,count
Items,
Coffee,5471
Bread,3325
Tea,1435
Cake,1025
Pastry,856
...,...
Bacon,1
Gift voucher,1
Olum & polenta,1


Widzimy, że w badanym okresie sprzedano wiele produktów z różną częstotliwością.

Wprowadzimy kategoryzację, dodając kolumnę **Item Categories**, która pozwoli nam wyróżnić 5 najpopularniejszych produktów, zaś pozostałym przypisać kategorię *Other*.

In [16]:
product_categories = list(bakery_data["Items"].value_counts().index)[0:5]
product_categories.append("Other")
product_categories
bakery_data["Item Categories"] = pd.Series(pd.Categorical(bakery_data["Items"], categories=product_categories)).fillna("Other")
bakery_data

,TransactionNo,Items,Date,Daypart,DayType,Item Categories
0,1,Bread,2016-10-30,Morning,Weekend,Bread
1,2,Scandinavian,2016-10-30,Morning,Weekend,Other
2,2,Scandinavian,2016-10-30,Morning,Weekend,Other
3,3,Hot chocolate,2016-10-30,Morning,Weekend,Other
4,3,Jam,2016-10-30,Morning,Weekend,Other
...,...,...,...,...,...,...
20502,9682,Coffee,2017-09-04,Afternoon,Weekend,Coffee
20503,9682,Tea,2017-09-04,Afternoon,Weekend,Tea
20504,9683,Coffee,2017-09-04,Afternoon,Weekend,Coffee
20505,9683,Pastry,2017-09-04,Afternoon,Weekend,Pastry


#### Konwersja pory dnia do kategorii

Przyjrzymy się bliżej zawartości kolumny **Daypart**.

In [17]:
bakery_data["Daypart"].value_counts()

,count
Daypart,
Afternoon,11569
Morning,8404
Evening,520
Night,14


W przypadku tej kolumny lista kategorii jest krótka - zależy nam jedynie, by na wizualizacji kolejność pór dnia odzwierciedlała ich kolejność występowania.

Zdefiniujemy nową kolumnę **Day Part** i ustalimy właściwą kolejność kategorii i zastąpimy nią dotychczasową kolumnę **Daypart**.

In [18]:
bakery_data["Day Part"] = pd.Series(pd.Categorical(bakery_data["Daypart"], categories=["Morning", "Afternoon", "Evening", "Night"]))
bakery_data = bakery_data[["TransactionNo", "Items", "Date", "Day Part", "DayType", "Item Categories"]]
bakery_data

,TransactionNo,Items,Date,Day Part,DayType,Item Categories
0,1,Bread,2016-10-30,Morning,Weekend,Bread
1,2,Scandinavian,2016-10-30,Morning,Weekend,Other
2,2,Scandinavian,2016-10-30,Morning,Weekend,Other
3,3,Hot chocolate,2016-10-30,Morning,Weekend,Other
4,3,Jam,2016-10-30,Morning,Weekend,Other
...,...,...,...,...,...,...
20502,9682,Coffee,2017-09-04,Afternoon,Weekend,Coffee
20503,9682,Tea,2017-09-04,Afternoon,Weekend,Tea
20504,9683,Coffee,2017-09-04,Afternoon,Weekend,Coffee
20505,9683,Pastry,2017-09-04,Afternoon,Weekend,Pastry


#### Konwersja typu dnia do kategorii

Przyjrzyjmy się bliżej zawartości kolumny **Day Type**.

In [19]:
bakery_data["DayType"].value_counts()

,count
DayType,
Weekday,12807
Weekend,7700


Podobnie jak w przypadku pory dnia, lista kategorii jest krótka - przygotujemy nową kolumnę **Day Type** w analogiczny sposób i usuniemy niepotrzebną kolumnę z zestawienia.

In [20]:
bakery_data["Day Type"] = pd.Series(pd.Categorical(bakery_data["DayType"], categories=["Weekday", "Weekend"]))
bakery_data = bakery_data[["TransactionNo", "Items", "Date", "Day Part","Day Type", "Item Categories"]]
bakery_data

,TransactionNo,Items,Date,Day Part,Day Type,Item Categories
0,1,Bread,2016-10-30,Morning,Weekend,Bread
1,2,Scandinavian,2016-10-30,Morning,Weekend,Other
2,2,Scandinavian,2016-10-30,Morning,Weekend,Other
3,3,Hot chocolate,2016-10-30,Morning,Weekend,Other
4,3,Jam,2016-10-30,Morning,Weekend,Other
...,...,...,...,...,...,...
20502,9682,Coffee,2017-09-04,Afternoon,Weekend,Coffee
20503,9682,Tea,2017-09-04,Afternoon,Weekend,Tea
20504,9683,Coffee,2017-09-04,Afternoon,Weekend,Coffee
20505,9683,Pastry,2017-09-04,Afternoon,Weekend,Pastry


### Stworzenie ramek danych wykorzystywanych w wizualizacji

Poza samym zbiorem **bakery_data** na zajęciach będziemy potrzebowali jeszcze kilku perspektyw dla celów wizualizacji.

#### Statystyki dzienne

Poniżej liczymy ile produktów i w ramach ilu transacji kupowano dziennie z podziałem na typ dnia.

In [21]:
items_daily = bakery_data[["Date","Day Type", "Items"]].groupby(["Date", "Day Type"]).count()
transactions_daily = bakery_data[["Date","Day Type", "TransactionNo"]].groupby(["Date", "Day Type"]).nunique()
daytype_statistics_daily = pd.merge(items_daily, transactions_daily, on=["Date", "Day Type"])
daytype_statistics_daily

<ipython-input-21-ae17c6ab33f8>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  items_daily = bakery_data[["Date","Day Type", "Items"]].groupby(["Date", "Day Type"]).count()
<ipython-input-21-ae17c6ab33f8>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  transactions_daily = bakery_data[["Date","Day Type", "TransactionNo"]].groupby(["Date", "Day Type"]).nunique()


Items  TransactionNo
Date       Day Type                      
2016-01-11 Weekday     150             77
           Weekend       0              0
2016-01-12 Weekday      83             47
           Weekend       0              0
2016-02-11 Weekday     164             83
...                    ...            ...
2017-12-01 Weekend       0              0
2017-12-02 Weekday       0              0
           Weekend     113             48
2017-12-03 Weekday       0              0
           Weekend     146             62

[318 rows x 2 columns]

Poniżej liczymy ile produktów i w ramach ilu transacji kupowano dziennie z podziałem na porę dnia.

In [22]:
items_daily = bakery_data[["Date","Day Part", "Items"]].groupby(["Date", "Day Part"]).count()
transactions_daily = bakery_data[["Date","Day Part", "TransactionNo"]].groupby(["Date", "Day Part"]).nunique()
daypart_statistics_daily = pd.merge(items_daily, transactions_daily, on=["Date", "Day Part"])
daypart_statistics_daily

<ipython-input-22-57361b2a6c5f>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  items_daily = bakery_data[["Date","Day Part", "Items"]].groupby(["Date", "Day Part"]).count()
<ipython-input-22-57361b2a6c5f>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  transactions_daily = bakery_data[["Date","Day Part", "TransactionNo"]].groupby(["Date", "Day Part"]).nunique()


Items  TransactionNo
Date       Day Part                       
2016-01-11 Morning       56             29
           Afternoon     92             46
           Evening        2              2
           Night          0              0
2016-01-12 Morning       31             17
...                     ...            ...
2017-12-02 Night          0              0
2017-12-03 Morning       32             15
           Afternoon    114             47
           Evening        0              0
           Night          0              0

[636 rows x 2 columns]

#### Statystyki kategorii

Na koniec zwracamy jeszcze liczbę produktów zakupionych w ramach transakcji z informacją o porze i typie dnia.

In [23]:
items_count = bakery_data[["TransactionNo", "Items"]].groupby(["TransactionNo"]).count()
transactions_data = pd.merge(pd.DataFrame(bakery_data[["TransactionNo", "Day Type", "Day Part"]].drop_duplicates()), items_count, on="TransactionNo")
transactions_data

,TransactionNo,Day Type,Day Part,Items
0,1,Weekend,Morning,1
1,2,Weekend,Morning,2
2,3,Weekend,Morning,3
3,4,Weekend,Morning,1
4,5,Weekend,Morning,3
...,...,...,...,...
9460,9680,Weekend,Afternoon,1
9461,9681,Weekend,Afternoon,4
9462,9682,Weekend,Afternoon,4
9463,9683,Weekend,Afternoon,2


## Zadania